## ![Intro slide: Some Python Identities](images/title_slide.png)

In [1]:
import sys; print(sys.version)

3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]


## `x.__len__()`  ?

## ≡ `len(x)`  !

In [2]:
class FixedSize:
    def __len__(self):
        return 42

In [3]:
fs = FixedSize()
assert len(fs) == fs.__len__() == 42

## `y.__contains__(x)`

## ≡ `x in y`

In [4]:
("BCD" in "ABCDEF", "ABCDEF".__contains__("BCD"))

(True, True)

In [5]:
class SearchableListOfLists:
    def __init__(self, *lists):
        self._values = lists
    def __contains__(self, value):
        return any(value in thing for thing in self._values)

In [6]:
sl = SearchableListOfLists([1, 2], [4, 5], [7, 8])
[(i, i in sl) for i in range(10)]

[(0, False),
 (1, True),
 (2, True),
 (3, False),
 (4, True),
 (5, True),
 (6, False),
 (7, True),
 (8, True),
 (9, False)]

## `items.__getitem__(key)`

## ≡ `items[key]`

In [7]:
class AllValuesAreOne:
    def __init__(self, value):
        self.value = value
    def __getitem__(self, key):
        print("Key:", key)
        return 42

In [8]:
mc = AllValuesAreOne(", world")
mc["hello"], mc["goodbye"]

Key: hello
Key: goodbye


(42, 42)

In [9]:
mc[1:]


Key: slice(1, None, None)


42

In [10]:
"abcdefghijklmnopqrstuvwxyz"[slice(1, None, None)]

'bcdefghijklmnopqrstuvwxyz'

In [11]:
mc[20:1:]

Key: slice(20, 1, None)


42

In [12]:
mc[20:1:-2]

Key: slice(20, 1, -2)


42

In [13]:
"abcdefghijklmnopqrstuvwxyz"[slice(20, 1, -2)]

'usqomkigec'

## `items.__setitem__(key, value)`

## ≡ `items[key] = value`

In [14]:
class SettableItems:
    def __init__(self, v):
        self._list = []
        self.v = v
    def __setitem__(self, key, value):
        self._list.append((self.v, key, value))
    def dump(self):
        for item in self._list:
            print(item)

In [15]:
s = SettableItems("@@")
s["hello"] = "World"
s["farewell"] = "Cruel world"
s.dump()

('@@', 'hello', 'World')
('@@', 'farewell', 'Cruel world')


### `(x.__class__).method(x, *args, **kw)`

## ≡ `x.method(*args, **kw)`

In [16]:
class MyClass1:
    def method(self, positional, keyword="world"):
        print(self, positional, keyword)

In [17]:
mc = MyClass1()
mc.method("hello")
mc.method("goodbye", keyword="cruel world")

<__main__.MyClass1 object at 0x7fc116bae790> hello world
<__main__.MyClass1 object at 0x7fc116bae790> goodbye cruel world


In [18]:
mc.__class__

__main__.MyClass1

In [19]:
mc.__class__.method

<function __main__.MyClass1.method(self, positional, keyword='world')>

In [20]:
mc.__class__.method(mc, "hello", "everybody!")

<__main__.MyClass1 object at 0x7fc116bae790> hello everybody!


In [21]:
len(fs), fs.__len__(), fs.__class__.__len__(fs), fs.__class__ is FixedSize

(42, 42, 42, True)

## `f.__call__(*args, **kwargs)`

## ≡ `f(*args, **kwargs)`

In [22]:
s1 = SettableItems.__call__("££")
s1[1] = 2
s1[3] = 4
s1.dump()

('££', 1, 2)
('££', 3, 4)


## Therefore ...

## `f__class__.__call__(f, *args, **kwargs)`

## ≡  `f(*args, **kwargs)`

In [23]:
def f(x, y):
    return x+y

f("Hello", "world")

'Helloworld'

In [24]:
f.__call__("Hello", "world")

'Helloworld'

In [25]:
f.__call__.__call__("Hello", "world")

'Helloworld'

In [26]:
f.__call__.__call__.__call__("Hello", "world")

'Helloworld'

In [27]:
f.__call__.__call__.__call__.__call__("Hello", "world")

'Helloworld'

In [28]:
f.__class__, type(f)

(function, function)

In [29]:
f.__class__.__call__(f, "Hello", "world")

'Helloworld'

In [30]:
SettableItems.__class__

type

In [31]:
s2 = type.__call__(SettableItems, "##")
s2[1] = "one"
s2[2] = "two"
s2.dump()

('##', 1, 'one')
('##', 2, 'two')


# And finally ...

## `X = type('X', (base1, base2, ...), { definitions_dict})`

## ≡ `class X(base1, base2, ...): ...`

In [32]:
def m(self, other):
    return self is other

In [33]:
T = type('T++', (object, ), {"is_same_as": m})
T

__main__.T++

In [34]:
t = T()
t

<__main__.T++ at 0x7fc116bc63d0>

In [35]:
t.is_same_as(None), t.is_same_as(t)

(False, True)

In [36]:
class print_args(type):
    def __new__(cls, name, bases, name_space):
        print("class:", cls)
        print("Declaring class", name)
        print("Bases:", bases)
        print("Namespace:", name_space)
        return type.__new__(cls, name, bases, name_space)

In [37]:
class A: pass

class B: pass

In [38]:
print("Defining C")
class C(A, B, metaclass=print_args):
    CVar = "class attribute"
    def myMethod(self, a):
        return self.N, self.CVar
    def __init__(self, N):
        self.N = N

Defining C
class: <class '__main__.print_args'>
Declaring class C
Bases: (<class '__main__.A'>, <class '__main__.B'>)
Namespace: {'__module__': '__main__', '__qualname__': 'C', 'CVar': 'class attribute', 'myMethod': <function C.myMethod at 0x7fc116b9db80>, '__init__': <function C.__init__ at 0x7fc116b9d310>}


In [39]:
print("Creating c")
c = C(42)
print("CVars:", C.CVar, c.CVar)
print("C:", C.__dict__)
print("c:", c.__dict__)

Creating c
CVars: class attribute class attribute
C: {'__module__': '__main__', 'CVar': 'class attribute', 'myMethod': <function C.myMethod at 0x7fc116b9db80>, '__init__': <function C.__init__ at 0x7fc116b9d310>, '__doc__': None}
c: {'N': 42}


In [40]:
c.CVar = "instance attribute"
print("CVars:", c.CVar, C.CVar)
print("c:", c.__dict__)

CVars: instance attribute class attribute
c: {'N': 42, 'CVar': 'instance attribute'}


### notebook available at

#### `https://github.com/steveholden/pydentities/`

![caption](images/tiny.jpg)